In [79]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [80]:
address = pd.read_csv('address.csv', sep=',')

In [81]:
address.head()

,id,town_ksh_code,province_name,town_name,town_type1,town_type2,town_type3,district_ksh_code,town_population,street_name,house_number,target
0,1,27872,13,Abony,3,2,3,124,14916,ATTILA UTCA,27,0
1,2,27872,13,Abony,3,2,3,124,14916,BICSKEI ÚT,4,1
2,3,27872,13,Abony,3,2,3,124,14916,CSIKY GERGELY UTCA,20,1
3,4,27872,13,Abony,3,2,3,124,14916,DR. HIRN MÁRTON UTCA,3,2
4,5,27872,13,Abony,3,2,3,124,14916,VASÚT ÚT,41,1


In [82]:
# Lowercase all the addresses
address['street_name'] = address['street_name'].str.lower()

In [ ]:
address['address'] = address['street_name'] + ' ' + address['house_number'].astype(str) + ', ' + address['town_name']

In [ ]:
# Requesting geocoding with ArcGIS API
def geocode(address):
    url = 'https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/findAddressCandidates'
    params = {'f': 'json', 'singleLine': address, 'outFields': 'Match_addr,Addr_type'}
    r = requests.get(url, params=params)
    if r.json()['candidates']:
        return r.json()['candidates'][0]['location']
    else:
        return None

In [ ]:
address['lon'] = address['address'].apply(geocode).apply(lambda x: x['x'] if x else None)

In [93]:
# Checking how many addresses were geocoded
address['lon'].isnull().sum()

0

In [94]:
address['lat'] = address['address'].apply(geocode).apply(lambda x: x['y'] if x else None)

In [95]:
# Checking how many addresses were geocoded
address['lat'].isnull().sum()

0

In [96]:
# Saving the data
address.to_csv('address_geocoded.csv', index=False)